In [ ]:
import numpy as numpy
import pandas as pd

In [ ]:
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies (1).csv")

In [ ]:
credits_df

In [ ]:
movies_df

In [ ]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [ ]:
credits_df

In [ ]:
movies_df

In [ ]:
credits_df.head()

In [ ]:
movies_df.tail()

In [ ]:
movies_df = movies_df.merge(credits_df, on = 'title')

In [ ]:
movies_df.shape

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df.isnull().sum()

In [ ]:
movies_df.dropna(inplace=True)

In [ ]:
movies_df.duplicated().sum()

In [ ]:
movies_df.iloc[0].genres

In [ ]:
import ast

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df.head()

In [ ]:

def convert3(obj):
    L = []
    counter = 0 
    if isinstance(obj, str) and obj.strip():
        try:
            cast_list = ast.literal_eval(obj)  
            for i in cast_list:
                if counter != 3:
                    L.append(i.get('name', 'Unknown'))  
                    counter += 1
                else:
                    break
        except (ValueError, SyntaxError):  
            return []
    return L

In [ ]:
movies_df['cast'] =  movies_df['cast'].apply(convert3)

In [ ]:
movies_df.head()

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)

In [ ]:
movies_df.head()

In [ ]:
movies_df['overview'][0]

In [ ]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [ ]:
movies_df.head()

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [ ]:
movies_df.head()

In [ ]:
movies_df['tags'] = movies_df['overview']+movies_df['genres']+movies_df['keywords']+movies_df['cast']+movies_df['crew']

In [ ]:
movies_df.head()

In [ ]:
new_df = movies_df[['movie_id', 'title', 'tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda X:X.lower())

In [ ]:
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
cv.fit_transform(new_df['tags']).toarray().shape

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y =[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
similarity[0].shape

In [ ]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]

In [ ]:
def recommend(moive):
    movie_index = new_df[new_df['title']==moive].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse =True, key = lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Iron Man')

In [ ]:
recommend('Liar Liar')

In [ ]:
recommend('Avengers: Age of Ultron')